<a href="https://colab.research.google.com/github/pastured3ton/Robot/blob/main/notebooks/forward_kinematics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import google.colab
%pip install roboticstoolbox-python>=1.0.2
%pip install colored==1.4.4
%pip install numpy==1.26.4
%pip install matplotlib==3.7

In [24]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})


In [25]:
L1 = 0.0
L2 = 0.5
L3 = 0.3
L4 = 0.01


The robotics toolbox allows you to define a robot in many ways.
Here the robot model will be based on 2 types of objects: **DHLink** and **DHRobot**. If you are using Matlab environment the same effect is available when using commands **Link** and **SeriaLink**.

Object **DHLink** can contain information about both kinematic and dynamic properties. Only elements related to the kinematics of the manipulator will be considered here. Defining a single **DHLink** object looks like this
```py
L1 = rtb.DHLink(d=0.0, alpha=0.0, theta=0.0, a=0.0, sigma=0)
```
where L1 is the name of the variable, into which the created object will be entered. The required parameters are in accordance with the DH notation, the additional parameter **sigma** defines the joint type: 0 - rotary joint, 1 - prismatic joint. The values given in the example above are the default values of individual parameters, which means that it is not necessary to provide the values of all parameters and if any parameter is missing, the default value will be used.
For example, a rotary joint can be defined like this:
```py
L1 = rtb.DHLink(d=1.0)
```
In Python, you can also omit the names of the parameters by giving arguments only, then their interpretation is determined by the order, you cannot omit parameters with a default value, e.g.:
```py
L1 = rtb.DHLink(1.0)                # equals L1 = rtb.DHLink(d=1.0)
L2 = rtb.DHLink(1.0, 0.0, 0.0, 0.5) # equals L1 = rtb.DHLink(d=1.0, a=0.5)
L2 = rtb.DHLink(1.0, 0.5)           # equals L1 = rtb.DHLink(d=1.0, alpha=0.5)
```

The creation of a simple two-joints manipulator may look like this:

In [26]:
# Joint 1: Prismatic (sigma=1)
J1 = rtb.DHLink(d=0.0, alpha=0.0, a=L1, theta=0.0, sigma=1)

# Joint 2: Revolute
J2 = rtb.DHLink(d=0.0, alpha=0.0, a=L2, theta=0.0, sigma=0)

# Joint 3: Revolute
J3 = rtb.DHLink(d=0.0, alpha=0.0, a=L3, theta=0.0, sigma=0)

# Joint 4: Revolute
J4 = rtb.DHLink(d=0.0, alpha=0.0, a=L4, theta=0.0, sigma=0)

To build a robot model, you need to create a new DHRobot object. As an argument to the constructor, it is enough to pass a list of defined objects of the DHLink type, which in the given order will create the kinematic chain of the manipulator.

In [27]:
# Build robot model
robot = rtb.DHRobot([J1, J2, J3, J4], name="Planar_TRRR")

After the robot object is created, you can view the DH table and validate the model.

In [28]:
print(robot)

DHRobot: Planar_TRRR, 4 joints (PRRR), dynamics, standard DH parameters
┌──────┬─────┬──────┬──────┐
│  θⱼ  │ dⱼ  │  aⱼ  │  ⍺ⱼ  │
├──────┼─────┼──────┼──────┤
│ 0.0° │  q1 │    0 │ 0.0° │
│  q2  │   0 │  0.5 │ 0.0° │
│  q3  │   0 │  0.3 │ 0.0° │
│  q4  │   0 │ 0.01 │ 0.0° │
└──────┴─────┴──────┴──────┘

┌──┬──┐
└──┴──┘



**Note: For rotary joints, the value of the variable angle θ has to be declared as 0, the same applies to the d parameter for prismatic joints.**

A robot created in this way can be visualized using the plot command invoked on a **DHRobot** object. As an argument, a list of values of successive join variables should be given.

In [29]:
robot.plot([0.0, 0.0, 0.0, 0.0])

PyPlot3D backend, t = 0.05, scene:
  robot: Text(0.0, 0.0, 'Planar_TRRR')

By giving an array of many elements as an argument to the **plot** method, it is possible to show an animation of the manipulator's movement.

In [30]:
qr = []
for i in range(101):
    # Provide values for all 4 joints. Assuming the last two joints are 0.
    q = [0.0, pi/100 * i, 0.0, 0.0]
    qr.append(q)

q_array = np.array(qr)
robot.plot(q_array)

PyPlot3D backend, t = 5.04999999999999, scene:
  robot: Text(0.0, 0.0, 'Planar_TRRR')

The position of the manipulator tip for the given values of joint variables - a solution of a forward kinematics problem - can be found using the **fkine** method. As in the **plot** method, the values of the join variables should be given as an argument.

In [31]:
robot.fkine([0.0, 0.0, 0.0, 0.0])

   1         0         0         0.81      
   0         1         0         0         
   0         0         1         0         
   0         0         0         1         


The returned result is a homogeneous matrix describing the location of the manipulator's tip in space.